In [1]:
!pip install -q tensorflow-recommenders

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import deque
#from surprise import Dataset, NormalPredictor, Reader
#from surprise.model_selection import cross_validate
import os
from sklearn.model_selection import train_test_split
import pickle
#from surprise import accuracy
#from surprise import accuracy, Dataset, Reader, SVD, KNNBaseline
#from surprise.model_selection import PredefinedKFold
import pprint
import tempfile
from typing import Dict, Text
import tensorflow as tf
import tensorflow_recommenders as tfrs
from collections import defaultdict
import joblib

In [ ]:
train_hr0 = pd.read_csv('rmse/hit_rate_folds_actual/train_hr0.csv', header=None)
test_hr0 = pd.read_csv('rmse/hit_rate_folds_actual/test_hr0.csv', header=None)

In [ ]:
train_hr0.columns = ['user_id', 'movie_id', 'rating']
test_hr0.columns = ['user_id', 'movie_id', 'rating']

In [8]:
top_ratings_df = pd.read_csv('rmse/top_ratings_df.csv')
top_ratings_df.head()

,user_id,movie_id,rating
0,841872,14454,5.0
1,689085,299,5.0
2,2529854,11165,5.0
3,2597445,10886,4.0
4,2336656,3282,5.0


In [11]:
top_ratings_df[top_ratings_df['user_id']==841872]['movie_id'].values[0]

14454

In [ ]:
hr_model_0a = joblib.load(open('model0a.pkl', 'rb'))

predictions_1a = defaultdict(list)

print('First dataframe inference loop')

for i in range(len(test_hr0)):
    user = test_hr0['user_id'].loc[i]
    movie = test_hr0['movie_id'].loc[i]
    predicted_rating = hr_model_0a({'user_id': np.array([user]),
                                           'movie_id': np.array([movie])})
    predictions_1a[user].append((movie, predicted_rating.numpy()[0][0]))
    
hr_model_0b = joblib.load(open('model0b.pkl', 'rb'))

predictions_1b = defaultdict(list)

print('Second dataframe inference loop')

for i in range(len(train_hr0)):
    user = train_hr0['user_id'].loc[i]
    movie = train_hr0['movie_id'].loc[i]
    predicted_rating = hr_model_0b({'user_id': np.array([user]),
                                           'movie_id': np.array([movie])})
    predictions_1b[user].append((movie, predicted_rating.numpy()[0][0]))
    
for user_id, user_ratings in predictions_1a.items():
    user_ratings.sort(key=lambda x: x[1], reverse=True)
    predictions_1a[user_id] = user_ratings[:100]
    
for user_id, user_ratings in predictions_1b.items():
    user_ratings.sort(key=lambda x: x[1], reverse=True)
    predictions_1b[user_id] = user_ratings[:100]

In [ ]:
num_users = 0
hits = 0

for user_id in predictions_1a.keys():
    top_movie = top_ratings_df[top_ratings_df['user_id']==user_id]['movie_id']
    num_users += 1
    num_recs = len(predictions_1a[user_id])
    top_number = int(num_recs*0.25)
    boundary_for_user = [rec[1] for rec in predictions_1a[user_id]][top_number]
    pred_top_movie = hr_model_0a({'user_id': np.array([user_id]),
                                         'movie_id': np.array([top_movie])})
    if pred_top_movie.numpy()[0][0] > boundary_for_user:
        hits += 1
        
hit_rate_1a = hits / num_users
print(hit_rate_1a)

In [ ]:
num_users = 0
hits = 0

for user_id in predictions_1b.keys():
    top_movie = top_ratings_df[top_ratings_df['user_id']==user_id]['movie_id']
    num_users += 1
    num_recs = len(predictions_1b[user_id])
    top_number = int(num_recs*0.25)
    boundary_for_user = [rec[1] for rec in predictions_1b[user_id]][top_number]
    pred_top_movie = hr_model_0b({'user_id': np.array([user_id]),
                                         'movie_id': np.array([top_movie])})
    if pred_top_movie.numpy()[0][0] > boundary_for_user:
        hits += 1
        
hit_rate_1b = hits / num_users
print(hit_rate_1b)

In [ ]:
hit_rates_nn = []


for i in range(5):
    hr_model_1a = joblib.load(open(f'model{i}a.pkl', 'rb'))
    hr_model_1b = joblib.load(open(f'model{i}b.pkl', 'rb'))
    train_hr = pd.read_csv(f'rmse/hit_rate_folds_actual/train_hr{i}.csv', header=None)
    test_hr = pd.read_csv(f'rmse/hit_rate_folds_actual/test_hr{i}.csv', header=None)
    train_hr.columns = ['user_id', 'movie_id', 'rating']
    test_hr.columns = ['user_id', 'movie_id', 'rating']
    
    predictions_a = defaultdict(list)
    
    print(f'First iteration {i}')
    
    for ii in range(len(test_hr)):
        user = test_hr['user_id'].loc[ii]
        movie = test_hr['movie_id'].loc[ii]
        predicted_rating = hr_model_1a({'user_id': np.array([user]),
                                           'movie_id': np.array([movie])})
        predictions_a[user].append((movie, predicted_rating.numpy()[0][0]))
        
    for user_id, user_ratings in predictions_a.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        predictions_a[user_id] = user_ratings[:100]
        
    predictions_b = defaultdict(list)
    
    print(f'Second iteration {i}')

    for iii in range(len(train_hr)):
        user = train_hr['user_id'].loc[iii]
        movie = train_hr['movie_id'].loc[iii]
        predicted_rating = hr_model_1b({'user_id': np.array([user]),
                                           'movie_id': np.array([movie])})
        predictions_b[user].append((movie, predicted_rating.numpy()[0][0]))
        
    
    for user_id, user_ratings in predictions_b.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        predictions_b[user_id] = user_ratings[:100]
        
    num_users_a = 0
    num_users_b = 0
    num_hits_a = 0
    num_hits_b = 0
    
    print(f'Calculating hit rate {i}')
    
    for user_id in predictions_a.keys():
        top_movie = top_ratings_df[top_ratings_df['user_id']==user_id]['movie_id'].values[0]
        num_users_a += 1
        num_recs = len(predictions_a[user_id])
        top_number = int(num_recs*0.25)
        boundary_for_user = [rec[1] for rec in predictions_a[user_id]][top_number]
        pred_top_movie = hr_model_1a({'user_id': np.array([user_id]),
                                         'movie_id': np.array([top_movie])})
        if pred_top_movie.numpy()[0][0] > boundary_for_user:
            num_hits_a += 1
            
    for user_id in predictions_b.keys():
        top_movie = top_ratings_df[top_ratings_df['user_id']==user_id]['movie_id'].values[0]
        num_users_b += 1
        num_recs = len(predictions_b[user_id])
        top_number = int(num_recs*0.25)
        boundary_for_user = [rec[1] for rec in predictions_b[user_id]][top_number]
        pred_top_movie = hr_model_1b({'user_id': np.array([user_id]),
                                         'movie_id': np.array([top_movie])})
        if pred_top_movie.numpy()[0][0] > boundary_for_user:
            num_hits_b += 1
            
    hit_rates_nn.append(num_hits_a / num_users_a)
    hit_rates_nn.append(num_hits_b / num_users_b)
    
    

First iteration 0
Second iteration 0
Calculating hit rate 0
First iteration 1
Second iteration 1
Calculating hit rate 1
First iteration 2
Second iteration 2
Calculating hit rate 2
First iteration 3
Second iteration 3
Calculating hit rate 3
First iteration 4
Second iteration 4
Calculating hit rate 4


In [13]:
hit_rates_nn

[0.4881216091016192,
 0.48702184750956684,
 0.4885011683712562,
 0.48638756829090685,
 0.48784170193297494,
 0.48601130684395716,
 0.4870200664816566,
 0.4862847930236724,
 0.48787433996073126,
 0.4867752090431654]

In [ ]:
train_rmse0 = pd.read_csv('rmse/rmse_folds_actual/actual_rmse_train0.csv', header=None)
test_rmse0 = pd.read_csv('rmse/rmse_folds_actual/actual_rmse_test0.csv', header=None)

train_rmse0.columns = ['user_id', 'movie_id', 'rating']
test_rmse0.columns = ['user_id', 'movie_id', 'rating']

rmse_model_0a = joblib.load(open('model_rmse0a.pkl', 'rb'))
rmse_model_0b = joblib.load(open('model_rmse0b.pkl', 'rb'))

preds_0a = []
preds_0b = []

rmse_nn = []

for i in range(len(test_rmse0)):
    user = test_rmse0['user_id'].loc[i]
    movie = test_rmse0['movie_id'].loc[i]
    predicted_rating = rmse_model_0a({'user_id': np.array([user]),
                                           'movie_id': np.array([movie])})
    preds_0a.append(predicted_rating.numpy()[0][0])
    
for i in range(len(train_rmse0)):
    user = train_rmse0['user_id'].loc[i]
    movie = train_rmse0['movie_id'].loc[i]
    predicted_rating = rmse_model_0b({'user_id': np.array([user]),
                                           'movie_id': np.array([movie])})
    preds_0b.append(predicted_rating.numpy()[0][0])
    
actual_ratings_0a = test_rmse0['rating'].values
actual_ratings_0b = train_rmse0['rating'].values

rmse_nn.append(np.sqrt(np.mean((actual_ratings_0a - np.array(preds_0a))**2)))
rmse_nn.append(np.sqrt(np.mean((actual_ratings_0b - np.array(preds_0b))**2)))

In [ ]:
rmse_nn = []

for i in range(5):
    train_rmse = pd.read_csv(f'rmse/rmse_folds_actual/actual_rmse_train{i}.csv', header=None)
    test_rmse = pd.read_csv(f'rmse/rmse_folds_actual/actual_rmse_test{i}.csv', header=None)
    train_rmse.columns = ['user_id', 'movie_id', 'rating']
    test_rmse.columns = ['user_id', 'movie_id', 'rating']
    rmse_model_a = joblib.load(open(f'model_rmse{i}a.pkl', 'rb'))
    rmse_model_b = joblib.load(open(f'model_rmse{i}b.pkl', 'rb'))
    
    preds_a = []
    preds_b = []
    
    for ii in range(len(test_rmse)):
        user = test_rmse['user_id'].loc[ii]
        movie = test_rmse['movie_id'].loc[ii]
        predicted_rating = rmse_model_a({'user_id': np.array([user]),
                                           'movie_id': np.array([movie])})
        preds_a.append(predicted_rating.numpy()[0][0])
        
    for iii in range(len(train_rmse)):
        user = train_rmse['user_id'].loc[iii]
        movie = train_rmse['movie_id'].loc[iii]
        predicted_rating = rmse_model_b({'user_id': np.array([user]),
                                           'movie_id': np.array([movie])})
        preds_b.append(predicted_rating.numpy()[0][0])
        
    
    rmse_nn.append(np.sqrt(np.mean((test_rmse['rating'].values - np.array(preds_a))**2)))
    rmse_nn.append(np.sqrt(np.mean((train_rmse['rating'].values - np.array(preds_b))**2)))
    

    

In [15]:
hr_nn_dict = dict(neural_network_hr=hit_rates_nn)
rmse_nn_dict = dict(neural_network_rmse=rmse_nn)

hr_nn_df = pd.DataFrame(hr_nn_dict)
rmse_nn_df = pd.DataFrame(rmse_nn_dict)

hr_nn_df
hr_nn_df.to_csv('hr_nn_df.csv', index=False)
rmse_nn_df.to_csv('rmse_nn_df.csv', index=False)